# Quiz 3: Nonlinear Programming


In [2]:
import pandas as pd
import numpy as np

import gurobipy as gb
from gurobipy import *

# Problem 1: The Markowitz Portfolio Optimization Problem (30 pts)

You have visited the Markowitz portfolio optimization problem in the class and Assignment 3. In this problem we want to solve a group of questions related to this problem. The setup of this problem (including the notion of risk as the **standard deviation** of the return of a portfolio) is similar to Problem 1 of Assignment 3:
1. We have five stocks and want to manage a portfolio of these stocks.
2. We have the average return of each stock, the covariance matrix of the returns of these stocks, and, just in case, the correlation matrix of these stocks. They are given in each question.
3. The investment amount in each stock is called **the weight of that stock**, and they construct our decision variables. The weights are non-negative (there is no short-selling,) and they must sum to 1.

## Question 1: Maximizing the expected return (10 pts)

In this question, we want to maximize the expected return of the portfolio subject to a 0.035 cap on the risk. That is, the standard deviation of the return of the portfolio should be less than or equal to 0.035. What is the **optimal portfolio, the optimal expected return, and the risk of the optimal portfolio**?

**Hint 1**: This question is similar to Question 4 of Assignment 3. So, you can use the same approach to solve this question. \
**Hint 2**: As we mentioned in the assignment, in solving these types of problems, working with the variance is easier than working with the standard deviation. A standard deviation $\leq 0.035$ is equivalent to a variance $\leq 0.035^2$.


In [12]:
Stock_Name = ['A', 'B', 'C', 'D', 'E']

RoR_Avg = [0.00809915, 0.00766259, 0.00819196, 0.00885855, 0.00792346]

RoR_Cov = [[0.00093265, 0.00094078, 0.00064858, 0.00070263, 0.00060319],
           [0.00094078, 0.00204336, 0.00088278, 0.00102208, 0.00107631],
           [0.00064858, 0.00088278, 0.0009473 , 0.00082019, 0.00097678],
           [0.00070263, 0.00102208, 0.00082019, 0.00167719, 0.00090011],
           [0.00060319, 0.00107631, 0.00097678, 0.00090011, 0.00162076]]

RoR_Corr = [[1.        , 0.68148801, 0.69002367, 0.56179557, 0.4906123 ],
            [0.68148801, 1.        , 0.63450648, 0.55210325, 0.59142981],
            [0.69002367, 0.63450648, 1.        , 0.65069868, 0.78830841],
            [0.56179557, 0.55210325, 0.65069868, 1.        , 0.54594033],
            [0.4906123 , 0.59142981, 0.78830841, 0.54594033, 1.        ]]

#############################################
# Create a new model.
model = gb.Model('Question1')
model.Params.LogToConsole = 0
m = len(Stock_Name)

# Create variables.
x = model.addVars(m, vtype = gb.GRB.CONTINUOUS, lb = 0, ub = 1, name = ['Weight_' + Stock_Name[i] for i in range(m)])

# Set objective function.
obj = gb.quicksum(RoR_Avg[i] * x[i] for i in range(m))
model.setObjective(obj, gb.GRB.MAXIMIZE)

# Set constraints.
## Constraint 1: The sum of the weights of all stocks must be 1.
model.addConstr(gb.quicksum(x[i] for i in range(m)) == 1, name = 'Sum of weights should be 1')

## Constraint 2: The risk of the portfolio must be less than 0.035.
model.addConstr(gb.quicksum(RoR_Cov[i][j] * x[i] * x[j] for i in range(m) for j in range(m)) <= 0.035**2, name = 'Risk should be less than 0.035')

# Optimize the model.
model.optimize()

# Print the optimal value of the decision variables.
for i in range(m):
    print(f'{x[i].VarName} = {round(x[i].x, 6)}')

# Print the optimal value of the objective function.
print(f'The optimal expected return is: {round(model.objVal, 6)}')

# Print the risk of the optimal portfolio.
print('The risk of the optimal portfolio is: ', round(np.sqrt(quicksum(x[i] * x[j] * RoR_Cov[i][j] for i in range(m) for j in range(m)).getValue()), 6))




Weight_A = 0.146233
Weight_B = 4e-06
Weight_C = 0.144293
Weight_D = 0.709456
Weight_E = 1.3e-05
The optimal expected return is: 0.008651
The risk of the optimal portfolio is:  0.034999


## Question 2: Maximizing the expected return with transaction costs (10 pts)

Question 5 of Assignment 3 discussed why considering a transaction cost is essential in portfolio optimization. In this question, we want to solve a question similar to Question 5 of the assignment but with a different structure. The transaction cost structure in this question is in the absolute value form rather than a quadratic form. That is, the objective function of this question is as follows:
$$\sum_{i = 1}^5 R_i \times x_i - 0.0005 \times \sum_{i = 1}^5 |x_i - 0.2|,$$
where $x_i$ is the weight of the *i*th index in the new portfolio, and $R_i$ is the average return of the *i*th index. Suppose we want to maximize the objective function subject to risk $\leq 0.035$. What is the **optimal portfolio, the optimal objective function, and the risk of the optimal portfolio**?

In [13]:
Stock_Name = ['A', 'B', 'C', 'D', 'E']

RoR_Avg = [0.00809915, 0.00766259, 0.00819196, 0.00885855, 0.00792346]

RoR_Cov = [[0.00093265, 0.00094078, 0.00064858, 0.00070263, 0.00060319],
           [0.00094078, 0.00204336, 0.00088278, 0.00102208, 0.00107631],
           [0.00064858, 0.00088278, 0.0009473 , 0.00082019, 0.00097678],
           [0.00070263, 0.00102208, 0.00082019, 0.00167719, 0.00090011],
           [0.00060319, 0.00107631, 0.00097678, 0.00090011, 0.00162076]]

RoR_Corr = [[1.        , 0.68148801, 0.69002367, 0.56179557, 0.4906123 ],
            [0.68148801, 1.        , 0.63450648, 0.55210325, 0.59142981],
            [0.69002367, 0.63450648, 1.        , 0.65069868, 0.78830841],
            [0.56179557, 0.55210325, 0.65069868, 1.        , 0.54594033],
            [0.4906123 , 0.59142981, 0.78830841, 0.54594033, 1.        ]]

#############################################
# Create a new model.
model = gb.Model('Question2')
model.Params.LogToConsole = 0
m = len(Stock_Name)

# Create variables.
x = model.addVars(m, vtype = gb.GRB.CONTINUOUS, lb = 0, ub = 1, name = ['Weight_' + Stock_Name[i] for i in range(m)])
z = model.addVars(m, vtype = gb.GRB.CONTINUOUS, lb = 0, ub = 1, name = ['Deviation_' + Stock_Name[i] for i in range(m)])

# Set objective function.
Exp_Return = gb.quicksum(RoR_Avg[i] * x[i] for i in range(m))
Total_Dev = gb.quicksum(z[i] for i in range(m))
model.setObjective(Exp_Return - 0.0005 * Total_Dev , gb.GRB.MAXIMIZE)

# Set constraints.
## Constraint 1: The sum of the weights of all stocks must be 1.
model.addConstr(gb.quicksum(x[i] for i in range(m)) == 1, name = 'Sum of weights should be 1')

## Constraint 2: The risk of the portfolio must be less than 0.035.
model.addConstr(gb.quicksum(RoR_Cov[i][j] * x[i] * x[j] for i in range(m) for j in range(m)) <= 0.035**2, name = 'Risk should be less than 0.035')

## Constraint 3: The deviation of each stock from 0.2:
model.addConstrs(z[i] >= x[i] - 0.2 for i in range(m))
model.addConstrs(z[i] >= 0.2 - x[i] for i in range(m))

# Optimize the model.
model.optimize()

# Print the optimal value of the decision variables.
for i in range(m):
    print(f'{x[i].VarName} = {round(x[i].x, 6)}')

# Print the optimal value of the objective function.
print(f'The optimal objective function is: {round(model.objVal, 6)}')

# Print the risk of the optimal portfolio.
print('The risk of the optimal portfolio is: ', round(np.sqrt(quicksum(x[i] * x[j] * RoR_Cov[i][j] for i in range(m) for j in range(m)).getValue()), 6))




Weight_A = 0.2
Weight_B = 1e-06
Weight_C = 0.2
Weight_D = 0.40001
Weight_E = 0.19999
The optimal objective function is: 0.008186
The risk of the optimal portfolio is:  0.031214


## Question 3: Discrete weights (10 pts)

We assumed the weights were continuous in the previous questions (and Assignment 3). In this question, we want to solve the same problem as Question 1 (maximizing the expected return subject to a risk cap) but with the additional constraint that **the weights should be multiples of 0.02**. That means the weight of each stock should be one of the numbers in $\{0, 0.02, 0.04, ..., 0.98, 1\}$. Subject to a risk $\leq 0.035$, and the constraint that the weights should be multiples of 0.02, maximize the expected return. What is the **optimal portfolio, the optimal expected return, and the risk of the optimal portfolio**?

**Hint**: Rounding the results to the closest multiple of 0.02 is not the correct answer. You should add some constraints to the model to ensure the weights are multiples of 0.02.

In [14]:
Stock_Name = ['A', 'B', 'C', 'D', 'E']

RoR_Avg = [0.00809915, 0.00766259, 0.00819196, 0.00885855, 0.00792346]

RoR_Cov = [[0.00093265, 0.00094078, 0.00064858, 0.00070263, 0.00060319],
           [0.00094078, 0.00204336, 0.00088278, 0.00102208, 0.00107631],
           [0.00064858, 0.00088278, 0.0009473 , 0.00082019, 0.00097678],
           [0.00070263, 0.00102208, 0.00082019, 0.00167719, 0.00090011],
           [0.00060319, 0.00107631, 0.00097678, 0.00090011, 0.00162076]]

RoR_Corr = [[1.        , 0.68148801, 0.69002367, 0.56179557, 0.4906123 ],
            [0.68148801, 1.        , 0.63450648, 0.55210325, 0.59142981],
            [0.69002367, 0.63450648, 1.        , 0.65069868, 0.78830841],
            [0.56179557, 0.55210325, 0.65069868, 1.        , 0.54594033],
            [0.4906123 , 0.59142981, 0.78830841, 0.54594033, 1.        ]]

#############################################
# Create a new model.
model = gb.Model('Question3')
model.Params.LogToConsole = 0
model.setParam('MIPGap', 0.00001)
m = len(Stock_Name)

# Create variables.
x = model.addVars(m, vtype = gb.GRB.CONTINUOUS, lb = 0, ub = 1, name = ['Weight_' + Stock_Name[i] for i in range(m)])
z = model.addVars(m, vtype = gb.GRB.INTEGER, lb = 0, ub = 50, name = ['Multiple of Weight_' + Stock_Name[i] for i in range(m)])

# Set objective function.
obj = gb.quicksum(RoR_Avg[i] * x[i] for i in range(m))
model.setObjective(obj, gb.GRB.MAXIMIZE)

# Set constraints.
## Constraint 1: The sum of the weights of all stocks must be 1.
model.addConstr(gb.quicksum(x[i] for i in range(m)) == 1, name = 'Sum of weights should be 1')

## Constraint 2: The risk of the portfolio must be less than 0.035.
model.addConstr(gb.quicksum(RoR_Cov[i][j] * x[i] * x[j] for i in range(m) for j in range(m)) <= 0.035**2, name = 'Risk should be less than 0.035')

## Constraint 3: Each weight should be a multiple of 0.02.
model.addConstrs(z[i] == x[i] * 50 for i in range(m))

# Optimize the model.
model.optimize()

# Print the optimal value of the decision variables.
for i in range(m):
    print(f'{x[i].VarName} = {round(x[i].x, 6)}')

# Print the optimal value of the objective function.
print(f'The optimal expected return is: {round(model.objVal, 6)}')

# Print the risk of the optimal portfolio.
print('The risk of the optimal portfolio is: ', round(np.sqrt(quicksum(x[i] * x[j] * RoR_Cov[i][j] for i in range(m) for j in range(m)).getValue()), 6))


Weight_A = 0.1
Weight_B = 0.0
Weight_C = 0.2
Weight_D = 0.7
Weight_E = 0.0
The optimal expected return is: 0.008649
The risk of the optimal portfolio is:  0.034972


# Problem 2: Balanced Milk and Nonlinear Taxes (25 pts)

Yep, you've read it correctly. We're discussing BalancedMilk's problems again! In the previous quiz, we solved the taxation problem of BalancedMilk with an interval taxation scheme. In this problem we want to solve the same problem with another nonlinear taxation scheme. In the following question, BalancedMilk's goal is to minimize the **Total Cost**, which is constructed from the transportation cost, not delivering cost, and the pollution tax:

**Total Cost = Transportation Cost + Not-Delivering Cost + Pollution Tax**

The transportation cost matrix and pollution matrix are provided in each question. In this problem, no demand market can receive more than its demand, and not delivering is also possible. For Questions 4 and 5, the not-delivering cost is 30 dollars per tonne. So, for instance, if BalancedMilk provides 100 tonnes of milk to $D_3$, the not-delivering cost due to this deficiency is $30 \times (150 - 100) = 1500$ dollars. The **Not-Delivering Cost** is the sum of all these costs for all demand markets. 



## Question 4: The Linear Taxation Scheme (5 pts)

Suppose the pollution tax is linear, and the tax rate is 3.5 dollars per Kg of pollution. Therefore, the pollution tax is:

Pollution Tax = $3.5 \times \text{Total Pollution}$

What is the **total amount of unsatisfied demand, the optimal total cost, and the amount of pollution tax**?

**Notice**: The total amount of unsatisfied demand is the sum of the gaps between the amount of milk each demand market receives and its demand.


In [5]:
Pollution = [[2, 4, 4, 5, 3, 5, 6, 2, 8, 4],
             [3, 7, 3, 4, 4.5, 2, 9, 1, 2, 5],
             [3, 4, 8, 3, 2, 2.5, 7, 2, 3, 8],
             [10, 5, 2, 5, 3, 5, 3, 6, 2, 7],
             [1, 1, 4, 5, 6, 2, 6, 2, 4, 3],
             [2, 1, 3, 4.5, 2, 8, 3, 6, 6, 1],
             [6, 7, 3, 3, 2, 6, 4, 6, 3, 10],
             [3, 3, 1, 5, 1, 5, 6, 3, 10, 4]]

Cost = [[35, 49, 16, 30,  15, 35, 12, 25, 10, 12],
        [15, 35, 7, 45, 47, 11, 16, 17, 15, 9],
        [22, 25, 42, 22, 31, 9, 11, 29, 36, 5],
        [32,  6, 40, 35, 11, 25, 30, 47, 32, 12],
        [9, 12, 19, 16, 38, 14, 53, 22, 19, 13],
        [21, 24, 32, 22, 5, 47, 30, 23, 19, 8],
        [43, 19,  5, 48, 47, 39, 15, 12, 9, 51],
        [34, 34, 10, 21, 9, 12, 14, 8, 19, 45]]

SupplyCenter = ['S_1', 'S_2', 'S_3', 'S_4', 'S_5', 'S_6', 'S_7', 'S_8']
Supply = [120, 100, 100, 260, 120, 250, 150, 500]
DemandMarket = ['D_1', 'D_2', 'D_3', 'D_4', 'D_5', 'D_6', 'D_7', 'D_8', 'D_9', 'D_10']
Demand = [100, 75, 150, 150, 180, 215, 170, 90, 160, 90]

#############################################
# Create a new model.
model = gb.Model('Question4')
model.Params.LogToConsole = 0
m = len(SupplyCenter)
n = len(DemandMarket)

# Create variables.
x = model.addVars(m, n, vtype = gb.GRB.CONTINUOUS, lb = 0, name = ['Supply_' + SupplyCenter[i] + '_to_' + DemandMarket[j] for i in range(m) for j in range(n)])

# Set objective function.
Transportation_Cost = gb.quicksum(Cost[i][j] * x[i, j] for i in range(m) for j in range(n))
Total_Pollution = gb.quicksum(Pollution[i][j] * x[i, j] for i in range(m) for j in range(n))
Not_Satisfied_Demand = sum(Demand) - quicksum(x[i, j] for i in range(m) for j in range(n))
model.setObjective(Transportation_Cost + 3.5 * Total_Pollution + 30 * Not_Satisfied_Demand, gb.GRB.MINIMIZE)

# Set constraints.
## Constraint 1: The total supply from each supply center must be less than or equal to its supply capacity.
model.addConstrs(gb.quicksum(x[i, j] for j in range(n)) <= Supply[i] for i in range(m))

## Constraint 2: The total milke delivered to each demand market cannot be more than its demand.
model.addConstrs(gb.quicksum(x[i, j] for i in range(m)) <= Demand[j] for j in range(n))

# Optimize the model.
model.optimize()

# Print the total unsatisfied demand.
print(f'The total unsatisfied demand is: {round(Not_Satisfied_Demand.getValue(), 2)} tonnes.')

# Print the optimal value of the objective function.
print(f'The optimal total cost is: ${round(model.objVal, 2)}')

# Print the total pollution tax.
print(f'The total pollution tax is: ${round(3.5 * Total_Pollution.getValue(), 2)}')


The total unsatisfied demand is: 330.0 tonnes.
The optimal total cost is: $26582.5
The total pollution tax is: $7822.5


## Question 5: The Convex Taxation Scheme (10 pts)

In this question, we assume as the pollution increases, the tax rate increases as well. That is the idea behind a convex taxation scheme. This way, by making the tax for high amounts of pollution unbearably high, the policymaker forces companies to move toward more efficient and less polluting technologies. In this question, we assume the tax rate is as follows:

Pollution Tax = $0.01 \times (\text{Total Pollution})^2$

What is **the total amount of unsatisfied demand, the optimal total cost, and the amount of pollution tax**?

In [6]:
Pollution = [[2, 4, 4, 5, 3, 5, 6, 2, 8, 4],
             [3, 7, 3, 4, 4.5, 2, 9, 1, 2, 5],
             [3, 4, 8, 3, 2, 2.5, 7, 2, 3, 8],
             [10, 5, 2, 5, 3, 5, 3, 6, 2, 7],
             [1, 1, 4, 5, 6, 2, 6, 2, 4, 3],
             [2, 1, 3, 4.5, 2, 8, 3, 6, 6, 1],
             [6, 7, 3, 3, 2, 6, 4, 6, 3, 10],
             [3, 3, 1, 5, 1, 5, 6, 3, 10, 4]]

Cost = [[35, 49, 16, 30,  15, 35, 12, 25, 10, 12],
        [15, 35, 7, 45, 47, 11, 16, 17, 15, 9],
        [22, 25, 42, 22, 31, 9, 11, 29, 36, 5],
        [32,  6, 40, 35, 11, 25, 30, 47, 32, 12],
        [9, 12, 19, 16, 38, 14, 53, 22, 19, 13],
        [21, 24, 32, 22, 5, 47, 30, 23, 19, 8],
        [43, 19,  5, 48, 47, 39, 15, 12, 9, 51],
        [34, 34, 10, 21, 9, 12, 14, 8, 19, 45]]

SupplyCenter = ['S_1', 'S_2', 'S_3', 'S_4', 'S_5', 'S_6', 'S_7', 'S_8']
Supply = [120, 100, 100, 260, 120, 250, 150, 500]
DemandMarket = ['D_1', 'D_2', 'D_3', 'D_4', 'D_5', 'D_6', 'D_7', 'D_8', 'D_9', 'D_10']
Demand = [100, 75, 150, 150, 180, 215, 170, 90, 160, 90]

#############################################
# Create a new model.
model = gb.Model('Question5')
model.Params.LogToConsole = 0
model.setParam('OptimalityTol', 0.001)
m = len(SupplyCenter)
n = len(DemandMarket)

# Create variables.
x = model.addVars(m, n, vtype = gb.GRB.CONTINUOUS, lb = 0, name = ['Supply_' + SupplyCenter[i] + '_to_' + DemandMarket[j] for i in range(m) for j in range(n)])

# Set objective function.
Transportation_Cost = gb.quicksum(Cost[i][j] * x[i, j] for i in range(m) for j in range(n))
Total_Pollution = gb.quicksum(Pollution[i][j] * x[i, j] for i in range(m) for j in range(n))
Not_Satisfied_Demand = sum(Demand) - quicksum(x[i, j] for i in range(m) for j in range(n))
model.setObjective(Transportation_Cost + 0.01 * Total_Pollution**2 + 30 * Not_Satisfied_Demand, gb.GRB.MINIMIZE)

# Set constraints.
## Constraint 1: The total supply from each supply center must be less than or equal to its supply capacity.
model.addConstrs(gb.quicksum(x[i, j] for j in range(n)) <= Supply[i] for i in range(m))

## Constraint 2: The total milke delivered to each demand market cannot be more than its demand.
model.addConstrs(gb.quicksum(x[i, j] for i in range(m)) <= Demand[j] for j in range(n))

# Optimize the model.
model.optimize()

# Print the total unsatisfied demand.
print(f'The total unsatisfied demand is: {round(Not_Satisfied_Demand.getValue(), 2)} tonnes.')

# Print the optimal value of the objective function.
print(f'The optimal total cost is: ${round(model.objVal, 2)}')

# Print the total pollution tax.
print(f'The total pollution tax is: ${round(0.01 * (Total_Pollution.getValue())**2, 2)}')


The total unsatisfied demand is: 750.0 tonnes.
The optimal total cost is: $32979.0
The total pollution tax is: $3969.0


## Question 6: Convex Taxation, Convex Not-Delivering Cost! (10 pts)

In this question, we use the same taxation scheme as Question 5. However, here, we consider a different not-delivering cost. In this question, the not-delivering cost **for each demand market** is as follows:

Not Delivering Cost for Demand Market i = $0.5 \times (\text{Market i's Demand - Milk Delivered to Market i})^2$

Naturally, the total not-delivering cost in the objective function is the sum of all these costs over all demand markets. Considering the convex taxation scheme of Question 5 and the convex not-delivering cost introduced above, what is **the total amount of unsatisfied demand, the optimal total cost, and the amount of pollution tax**?

In [7]:
Pollution = [[2, 4, 4, 5, 3, 5, 6, 2, 8, 4],
             [3, 7, 3, 4, 4.5, 2, 9, 1, 2, 5],
             [3, 4, 8, 3, 2, 2.5, 7, 2, 3, 8],
             [10, 5, 2, 5, 3, 5, 3, 6, 2, 7],
             [1, 1, 4, 5, 6, 2, 6, 2, 4, 3],
             [2, 1, 3, 4.5, 2, 8, 3, 6, 6, 1],
             [6, 7, 3, 3, 2, 6, 4, 6, 3, 10],
             [3, 3, 1, 5, 1, 5, 6, 3, 10, 4]]

Cost = [[35, 49, 16, 30,  15, 35, 12, 25, 10, 12],
        [15, 35, 7, 45, 47, 11, 16, 17, 15, 9],
        [22, 25, 42, 22, 31, 9, 11, 29, 36, 5],
        [32,  6, 40, 35, 11, 25, 30, 47, 32, 12],
        [9, 12, 19, 16, 38, 14, 53, 22, 19, 13],
        [21, 24, 32, 22, 5, 47, 30, 23, 19, 8],
        [43, 19,  5, 48, 47, 39, 15, 12, 9, 51],
        [34, 34, 10, 21, 9, 12, 14, 8, 19, 45]]

SupplyCenter = ['S_1', 'S_2', 'S_3', 'S_4', 'S_5', 'S_6', 'S_7', 'S_8']
Supply = [120, 100, 100, 260, 120, 250, 150, 500]
DemandMarket = ['D_1', 'D_2', 'D_3', 'D_4', 'D_5', 'D_6', 'D_7', 'D_8', 'D_9', 'D_10']
Demand = [100, 75, 150, 150, 180, 215, 170, 90, 160, 90]

#############################################
# Create a new model.
model = gb.Model('Question6')
model.Params.LogToConsole = 0
model.setParam('OptimalityTol', 0.01)
m = len(SupplyCenter)
n = len(DemandMarket)

# Create variables.
x = model.addVars(m, n, vtype = gb.GRB.CONTINUOUS, lb = 0, name = ['Supply_' + SupplyCenter[i] + '_to_' + DemandMarket[j] for i in range(m) for j in range(n)])

# Set objective function.
Transportation_Cost = gb.quicksum(Cost[i][j] * x[i, j] for i in range(m) for j in range(n))
Total_Pollution = gb.quicksum(Pollution[i][j] * x[i, j] for i in range(m) for j in range(n))
Not_Sat_Demand_Cost = 0.5 * gb.quicksum((Demand[j] - gb.quicksum(x[i, j] for i in range(m)))**2 for j in range(n))
model.setObjective(Transportation_Cost + 0.01 * Total_Pollution**2 + Not_Sat_Demand_Cost, gb.GRB.MINIMIZE)

# Set constraints.
## Constraint 1: The total supply from each supply center must be less than or equal to its supply capacity.
model.addConstrs(gb.quicksum(x[i, j] for j in range(n)) <= Supply[i] for i in range(m))

## Constraint 2: The total milke delivered to each demand market cannot be more than its demand.
model.addConstrs(gb.quicksum(x[i, j] for i in range(m)) <= Demand[j] for j in range(n))

# Optimize the model.
model.optimize()

# Print the total unsatisfied demand.
print(f'The total unsatisfied demand is: {round(sum(Demand) - quicksum(x[i, j] for i in range(m) for j in range(n)).getValue(), 2)} tonnes.')

# Print the optimal value of the objective function.
print(f'The optimal total cost is: ${round(model.objVal, 2)}')

# Print the total pollution tax.
print(f'The total pollution tax is: ${round(0.01 * (Total_Pollution.getValue())**2, 2)}')


The total unsatisfied demand is: 620.29 tonnes.
The optimal total cost is: $48640.74
The total pollution tax is: $15128.22


# Problem 3: Using Decision Analytics in the Daily Life! (45 pts)

In this problem, we want to see how a student (in this case, Alireza) can implement the ideas he learned from the Decision Analytics course in his academic life. Suppose Alireza is a student who is taking six courses this semester. It is mid-November, and he has three weeks left until the final exams. Based on his calculations, he has 100 hours to study for the exams. He wants to allocate this time to the courses as efficiently as possible. In all the questions, we assume that the passing grade for each course is 50 out of 100, and each course has three credits. Additionally, consider the time allocated to each course is a **continuous** variable.

## Question 7: Constant Return to Study - The Basic Case (10 pts)

Suppose Alireza has a constant **return to study**. This means that if he studies for $q$ hours for a course with a return of $r$ points per hour, his grade in that course will increase by $r.q$ points. The following table represents his current situation in each course:

| Course | Min Grade | Max Grade | Return to Study (points/hour) | Type |
|--------|---------------|-----------|-----------------| ---- |
| Machine Learning | 30 | 100 | 3 | Practical |
| Statistics | 43 | 100 | 1 | Practical |
| Econometrics | 35 | 100 | 2 | Practical |
| Microeconomics | 37 | 100 | 2 | Theoretical |
| Optimization | 30 | 100 | 2 | Theoretical |
| Game Theory | 35 | 100 | 3 | Theoretical |

The **Min Grade** in the above table is the grade Alireza will get without studying for that course. For example, if he allocates 10 hours to studying Game Theory, his grade for Game Theory will be $35 + 10 \times 3 = 65$. Alireza wants to allocate his 100 hours to the courses to maximize his total grade without failing any course. The total grade is the average of the grades in all the courses. For example, if Alireza does not study at all, his total grade will be $(30 + 43 + 35 + 37 + 30 + 35)/6 = 35$.

By coding or by hand, find the optimal allocation of time to each course that maximizes Alireza's total grade so that he does not fail any course. Report **the optimal time allocation and the maximum total grade**.

In [8]:
Course_Name = ['Machine Learning', 'Statistics', 'Econometrics', 'Microeconomics', 'Optimization', 'Game Theory']
Min_Grade = [30, 43, 35, 37, 30, 35]
Max_Grade = [100, 100, 100, 100, 100, 100]
Return_to_Study = [3, 1, 2, 2, 2, 3]
Type_Practical = [1, 1, 1, 0, 0, 0]

#############################################
# Create a new model.
model = gb.Model('Question7')
model.Params.LogToConsole = 0
model.setParam('OptimalityTol', 0.001)
m = len(Min_Grade)

# Create variables.
x = model.addVars(m, vtype = gb.GRB.CONTINUOUS, name = ['Time Allocated to Course ' + Course_Name[i] for i in range(m)])
y = model.addVars(m, vtype = gb.GRB.CONTINUOUS, name = ['Grade of Course ' + Course_Name[i] + ' is taken' for i in range(m)])

# Set objective function.
obj = (1/6) * gb.quicksum(y[i] for i in range(m))
model.setObjective(obj, gb.GRB.MAXIMIZE)

# Set constraints.
## Constraint 1: The grade for each course is relevant to the time allocated to it.
model.addConstrs(y[i] == Min_Grade[i] + Return_to_Study[i] * x[i] for i in range(m))

## Constraint 2: The total time allocated to all courses must be less than or equal to 100.
model.addConstr(gb.quicksum(x[i] for i in range(m)) <= 100)

## Constraint 3: No course can get a grade higher than 100.
model.addConstrs(y[i] <= Max_Grade[i] for i in range(m))

## Constraint 4: Alireza should not fail any course.
model.addConstrs(y[i] >= 50 for i in range(m))

# Optimize the model.
model.optimize()

# Print the optimal allocation of time to each course.
for i in range(m):
    print(f'{x[i].VarName} = {round(x[i].x, 2)} hrs')

# Print the optimal total grade.
print(f'The optimal total grade is: {round(model.ObjVal, 3)}')


Time Allocated to Course Machine Learning = 23.33 hrs
Time Allocated to Course Statistics = 7.0 hrs
Time Allocated to Course Econometrics = 7.5 hrs
Time Allocated to Course Microeconomics = 6.5 hrs
Time Allocated to Course Optimization = 34.0 hrs
Time Allocated to Course Game Theory = 21.67 hrs
The optimal total grade is: 74.667


## Question 8: Constant Return to Study - Some Additional Constraints

Alireza wants to add some additional constraints to the previous problem:

1. His average grade in the practical courses should be at least 65 ($\geq 65$,) and his average in the theoretical courses should be at least 60 ($\geq 60$.)
2. Either he should get at least 70 in Machine Learning, or at least 65 in Statistics, or both.
3. The gap between his highest and lowest grades should be at most 20.

### Question 8.1: Formulating the Additional Constraints (7.5 pts)

Formulate the additional constraints in the previous problem as mixed integer programming logical constraints. You can use the following variables as given: \
$Y_{ML}$: Alireza's grade in Machine Learning. \
$Y_{ST}$: Alireza's grade in Statistics. \
$Y_{EC}$: Alireza's grade in Econometrics. \
$Y_{MI}$: Alireza's grade in Microeconomics. \
$Y_{OP}$: Alireza's grade in Optimization. \
$Y_{GT}$: Alireza's grade in Game Theory. 

**Constraint 1. His average grade in the practical courses ...** \
1/3 (Y_{ML} + Y_{ST} + Y_{EC}) >= 65 \
1/3 (Y_{MI} + Y_{OP} + Y_{GT}) >= 60 \

**Constraint 2. Either he gets more than 70 ...** \
X: A binary variable, M: a large number \
$Y_{ML} \geq 70 - M \times X$ \
$Y_{ST} \geq 65 - M \times (1 - X)$ \

**Constraint 3. The gap between his highest grade and his lowest grade ...** \
U: A continuous variable, L: A continuous variable \
$U \geq Y_{i}$ for all i \
$L \leq Y_{i}$ for all i \
$U - L \leq 20$ \

### Question 8.2: Coding the Problem (7.5 pts)

Find the optimal time allocation to each course that maximizes Alireza's total grade such that he does not fail any course and meets the three additional constraints. Report **the optimal time allocation and the maximum total grade**

In [9]:
Course_Name = ['Machine Learning', 'Statistics', 'Econometrics', 'Microeconomics', 'Optimization', 'Game Theory']
Min_Grade = [30, 43, 35, 37, 30, 35]
Max_Grade = [100, 100, 100, 100, 100, 100]
Return_to_Study = [3, 1, 2, 2, 2, 3]
Type_Practical = [1, 1, 1, 0, 0, 0]

#############################################
# Create a new model.
model = gb.Model('Question8')
model.Params.LogToConsole = 0
m = len(Min_Grade)

# Create variables.
x = model.addVars(m, vtype = gb.GRB.CONTINUOUS, name = ['Time Allocated to Course ' + Course_Name[i] for i in range(m)])
y = model.addVars(m, vtype = gb.GRB.CONTINUOUS, name = ['Grade of Course ' + Course_Name[i] + ' is taken' for i in range(m)])
z = model.addVar(vtype=GRB.BINARY, name='Additional Constraint 2')
M = 100000
u = model.addVar(vtype=GRB.CONTINUOUS, name='Upper Bound for grades')
l = model.addVar(vtype=GRB.CONTINUOUS, name='Lower Bound for grades')

# Set objective function.
obj = (1/6) * gb.quicksum(y[i] for i in range(m))
model.setObjective(obj, gb.GRB.MAXIMIZE)

# Set constraints.
## Constraint 1: The grade for each course is relevant to the time allocated to it.
model.addConstrs(y[i] == Min_Grade[i] + Return_to_Study[i] * x[i] for i in range(m))

## Constraint 2: The total time allocated to all courses must be less than or equal to 100.
model.addConstr(gb.quicksum(x[i] for i in range(m)) <= 100)

## Constraint 3: No course can get a grade higher than 100.
model.addConstrs(y[i] <= Max_Grade[i] for i in range(m))

## Constraint 4: Alireza should not fail any course.
model.addConstrs(y[i] >= 50 for i in range(m))

#############################################
# Additional constraints.
## Average in Practical Courses must be at least 65.
model.addConstr((y[0] + y[1] + y[2]) / 3 >= 65)
## Average in Theoretical Courses must be at least 60.
model.addConstr((y[3] + y[4] + y[5]) / 3 >= 60)

## Either more than 70 in ML or 65 in Statistics.
model.addConstr(y[0] >= 70 - M * z)
model.addConstr(y[1] >= 65 - M * (1 - z))

## Upper bound and lower bound gap must be less than 20.
model.addConstrs(y[i] <= u for i in range(m))
model.addConstrs(y[i] >= l for i in range(m))
model.addConstr(u - l <= 20)

# Optimize the model.
model.optimize()

# Print the optimal allocation of time to each course.
for i in range(m):
    print(f'{x[i].VarName} = {round(x[i].x, 2)}')

# Print the optimal total grade.
print(f'The optimal total grade is: {round(model.ObjVal, 2)}')

# Print the optimal grades for each course.
for i in range(m):
    print(f'{y[i].VarName} = {round(y[i].x, 2)}')


Time Allocated to Course Machine Learning = 14.81
Time Allocated to Course Statistics = 11.42
Time Allocated to Course Econometrics = 19.71
Time Allocated to Course Microeconomics = 18.71
Time Allocated to Course Optimization = 22.21
Time Allocated to Course Game Theory = 13.14
The optimal total grade is: 71.09
Grade of Course Machine Learning is taken = 74.42
Grade of Course Statistics is taken = 54.42
Grade of Course Econometrics is taken = 74.42
Grade of Course Microeconomics is taken = 74.42
Grade of Course Optimization is taken = 74.42
Grade of Course Game Theory is taken = 74.42


## Question 9: Decreasing Return to Study (10 pts)

In reality, the return to study is not constant. We all have the experience that increasing our grades from 50% to 60% is much easier than increasing them from 90% to 100%. In this problem, we want to model this phenomenon. We assume that the grade in course i is given by the following formula:
$$
Y_{i} = y_{i0} + r_i \times \sqrt{q_i},
$$
where $Y_i$ is the grade Alireza gets in course $i$, $y_{i0}$ is the min grade Alireza gets in course $i$ without studying, $r_i$ is the return to study in course $i$, and $q_i$ is the number of hours Alireza studies for course $i$. The following table represents the values of $y_{i0}$ and $r_i$ for each course:

| Course | Min Grade | Max Grade | Return to Study | Type |
|--------|---------------|-----------|-----------------| ---- |
| Machine Learning | 30 | 100 | 12 | Practical |
| Statistics | 43 | 100 | 4 | Practical |
| Econometrics | 35 | 100 | 8 | Practical |
| Microeconomics | 37 | 100 | 8 | Theoretical |
| Optimization | 30 | 100 | 8 | Theoretical |
| Game Theory | 35 | 100 | 12 | Theoretical |

For example, if Alireza studies 16 hours for Machine Learning, his grade in the Machine Learning course will be $30 + 12 \times \sqrt{16} = 78$. Like Question 7, find the optimal allocation of time to each course that maximizes Alireza's total grade so that he does not fail any course. Do not implement the constraints introduced in Question 8. Report **the optimal time allocation and the maximum total grade**.

In [10]:
Course_Name = ['Machine Learning', 'Statistics', 'Econometrics', 'Microeconomics', 'Optimization', 'Game Theory']
Min_Grade = [30, 43, 35, 37, 30, 35]
Max_Grade = [100, 100, 100, 100, 100, 100]
Return_to_Study = [12, 4, 8, 8, 8, 12]
Type_Practical = [1, 1, 1, 0, 0, 0]

#############################################
# Create a new model.
model = gb.Model('Question9')
model.Params.LogToConsole = 0
model.params.NonConvex = 2
model.setParam('OptimalityTol', 0.001)
m = len(Min_Grade)

# Create variables.
x = model.addVars(m, vtype = gb.GRB.CONTINUOUS, name = ['Time Allocated to Course ' + Course_Name[i] for i in range(m)])
y = model.addVars(m, vtype = gb.GRB.CONTINUOUS, name = ['Grade of Course ' + Course_Name[i] + ' is taken' for i in range(m)])
x_sqrt = model.addVars(m, vtype = gb.GRB.CONTINUOUS, name = ['Square Root of Time Allocated to Course ' + Course_Name[i] for i in range(m)])

# Set objective function.
obj = (1/6) * gb.quicksum(y[i] for i in range(m))
model.setObjective(obj, gb.GRB.MAXIMIZE)

# Set constraints.
## Constraint 1: The grade for each course is relevant to the time allocated to it.
model.addConstrs(x[i] == x_sqrt[i] * x_sqrt[i] for i in range(m))
model.addConstrs(y[i] == Min_Grade[i] + Return_to_Study[i] * x_sqrt[i] for i in range(m))

## Constraint 2: The total time allocated to all courses must be less than or equal to 100.
model.addConstr(gb.quicksum(x[i] for i in range(m)) <= 100)

## Constraint 3: No course can get a grade higher than 100.
model.addConstrs(y[i] <= Max_Grade[i] for i in range(m))

## Constraint 4: Alireza should not fail any course.
model.addConstrs(y[i] >= 50 for i in range(m))

# Optimize the model.
model.optimize()

# Print the optimal allocation of time to each course.
for i in range(m):
    print(f'{x[i].VarName} = {round(x[i].x, 2)}')

# Print the optimal total grade.
print(f'The optimal total grade is: {round(model.ObjVal, 2)}')


Time Allocated to Course Machine Learning = 29.03
Time Allocated to Course Statistics = 3.23
Time Allocated to Course Econometrics = 12.9
Time Allocated to Course Microeconomics = 12.9
Time Allocated to Course Optimization = 12.9
Time Allocated to Course Game Theory = 29.03
The optimal total grade is: 72.12


## Question 10: Decreasing Return to Study - Choosing between Passion and Grades (10 pts)

Alireza is into climbing, and, in normal circumstances, he goes climbing for two sessions of two hours every week. However, the 100 hours he allocated to studying in the previous questions is based on **one climbing session per week**. That is, for those three weeks, he will visit the gym only once per week and for two hours. He loves climbing, and he has a climbing competition in two months. Naturally, he is wondering if he can allocate more time to climbing. 

Alireza wants to know if, for the next three weeks, instead of one session of two hours per week, he can go climbing for two sessions of two hours per week. In his mind, the joy of one additional two-hour climbing session per week equals 1.5 extra points in his total grade. That is, if one additional climbing session per week (which results in less time for studying) decreases his total grade by at most 1.5 points, he is willing to add the additional climbing session to his schedule.

Based on the setting for Question 9, should he change his plan? If yes, what would be his new total grade? If not, why not?

In [6]:
Course_Name = ['Machine Learning', 'Statistics', 'Econometrics', 'Microeconomics', 'Optimization', 'Game Theory']
Min_Grade = [30, 43, 35, 37, 30, 35]
Max_Grade = [100, 100, 100, 100, 100, 100]
Return_to_Study = [12, 4, 8, 8, 8, 12]
Type_Practical = [1, 1, 1, 0, 0, 0]

#############################################
# Create a new model.
model = gb.Model('Question9')
model.Params.LogToConsole = 0
model.params.NonConvex = 2
model.setParam('OptimalityTol', 0.001)
m = len(Min_Grade)

# Create variables.
x = model.addVars(m, vtype = gb.GRB.CONTINUOUS, name = ['Time Allocated to Course ' + Course_Name[i] for i in range(m)])
y = model.addVars(m, vtype = gb.GRB.CONTINUOUS, name = ['Grade of Course ' + Course_Name[i] + ' is taken' for i in range(m)])
x_sqrt = model.addVars(m, vtype = gb.GRB.CONTINUOUS, name = ['Square Root of Time Allocated to Course ' + Course_Name[i] for i in range(m)])
climbing = model.addVar(vtype=GRB.BINARY, name='Additional Climbing Session')

# Set objective function.
obj = (1/6) * gb.quicksum(y[i] for i in range(m)) + 1.5 * climbing
model.setObjective(obj, gb.GRB.MAXIMIZE)

# Set constraints.
## Constraint 1: The grade for each course is relevant to the time allocated to it.
model.addConstrs(x[i] == x_sqrt[i] * x_sqrt[i] for i in range(m))
model.addConstrs(y[i] == Min_Grade[i] + Return_to_Study[i] * x_sqrt[i] for i in range(m))

## Constraint 2: The total time allocated to all courses must be less than or equal to 100 (- climbing time).
model.addConstr(gb.quicksum(x[i] for i in range(m)) <= 100 - 6 * climbing)

## Constraint 3: No course can get a grade higher than 100.
model.addConstrs(y[i] <= Max_Grade[i] for i in range(m))

## Constraint 4: Alireza should not fail any course.
model.addConstrs(y[i] >= 50 for i in range(m))

# Optimize the model.
model.optimize()

# Print the optimal allocation of time to each course.
for i in range(m):
    print(f'{x[i].VarName} = {round(x[i].x, 2)}')
    
# Additional climbing session?
print(f'{climbing.VarName} = {climbing.x}')

# Print the optimal total grade.
print(f'The optimal total grade is: {round(((1/6) * gb.quicksum(y[i] for i in range(m))).getValue(), 2)}')

# Print the objective value.
print(f'The objective value is: {round(model.ObjVal, 2)}')


Time Allocated to Course Machine Learning = 27.28
Time Allocated to Course Statistics = 3.06
Time Allocated to Course Econometrics = 12.12
Time Allocated to Course Microeconomics = 12.12
Time Allocated to Course Optimization = 12.12
Time Allocated to Course Game Theory = 27.28
Additional Climbing Session = 1.0
The optimal total grade is: 70.99
The objective value is: 72.49


He prefers adding the additional climbing session because he sets *climbing = 1* in the optimal allocation. Notice that his total grade in Question 9 (i.e., without the extra session) is 72.12, and his new total grade (with the additional session) is 70.99. Therefore, since $70.99 + 1.5 > 72.12$, he is willing to add the extra session at the expense of a tiny (less than 1.5 pts) reduction in his grade. 